## 데이터

In [1]:
# 셀레니움 잘 깔렸는지 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.10.18
4.33.0


In [2]:
import os
import requests
import pandas as pd
import sqlite3
import re
import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller  # setup chrome options

import urllib.request
from urllib.request import urlretrieve
import time

import warnings
warnings.filterwarnings('ignore')

In [3]:
market_df = pd.read_csv('./markets_review_url_1296.csv', encoding='utf-8-sig')

In [4]:
market_df = market_df.loc[:, ~market_df.columns.str.contains('^Unnamed')]

In [5]:
market_df.head()

,mID,naverURL
0,경기도 가평군 설악눈메골시장,https://pcmap.place.naver.com/restaurant/14187...
1,경기도 가평군 가평잣고을시장,https://pcmap.place.naver.com/restaurant/38738...
2,경기도 가평군 청평여울시장,https://pcmap.place.naver.com/restaurant/11536...
3,경기도 고양시 원당시장,https://pcmap.place.naver.com/restaurant/32892...
4,경기도 고양시 능곡시장,https://pcmap.place.naver.com/restaurant/16192...


In [6]:
market_df['rating'] = ""

In [7]:
market_df.head()

,mID,naverURL,rating
0,경기도 가평군 설악눈메골시장,https://pcmap.place.naver.com/restaurant/14187...,
1,경기도 가평군 가평잣고을시장,https://pcmap.place.naver.com/restaurant/38738...,
2,경기도 가평군 청평여울시장,https://pcmap.place.naver.com/restaurant/11536...,
3,경기도 고양시 원당시장,https://pcmap.place.naver.com/restaurant/32892...,
4,경기도 고양시 능곡시장,https://pcmap.place.naver.com/restaurant/16192...,


# 프로세스

In [20]:
 # chrome_path = "/content/drive/MyDrive/1_public/chromedriver" # chromedirver 경로 알려주기

# 네이버 지도 리뷰

In [21]:
# market_df = market_df.rename(columns={
#     '시장ID': 'mID',
# })

In [8]:
# temp_df = pd.read_csv('markets_review_url.csv', encoding='utf-8-sig')
# temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]
# temp_df

In [7]:
# temp_df = temp_df[temp_df['naverURL'].notna() & (temp_df['naverURL'].str.strip() != '')].reset_index(drop=True)
# temp_df

In [45]:
# temp_df.to_csv('./markets_review_url_1296.csv', encoding='utf-8-sig', index=False)

In [8]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import re

# #지역명이 포함된 식당명을 변수로 지정
# items = market_df['mID']
# print(items)

# #검색할 식당 데이터와 url을 담을 데이터 프레임 생성
# df = pd.DataFrame(columns=['mID', 'naverURL'])

# #데이터 프레임이 잘 만들어졌는지 확인
# df['mID'] = items
# df

df = market_df.copy()

# 식당 url 얻기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('--disable-images')  # 이미지 로딩 비활성화
options.add_argument('--disable-javascript')  # 불필요한 JS 비활성화
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
options.add_argument("--no-sandbox")              # 샌드박스 비활성화
options.add_argument("--disable-dev-shm-usage")   # /dev/shm 사용 방지
options.add_argument("--disable-gpu")             # GPU 비활성화
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5)   # 최대 5초까지 기다림

# User-Agent 설정으로 봇 감지 회피
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [9]:
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

In [10]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [11]:
## 별점 rating
rev_list = []

for i, keyword in enumerate(df['mID'].tolist()):
    print(f"[{i+1}] {keyword} 검색 중...")

    try:

        # 검색 url 만들기
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
    
        driver.get(naver_map_search_url)
        
        # css를 찾을때 까지 10초 대기
        time_wait(10, 'div.input_box > input.input_search')
        
        # (1) 검색창 찾기
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(keyword)  # 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
        time.sleep(2) 
        
        # 검색 프레임 변경
        driver.switch_to.frame("searchIframe")
        time.sleep(1)
        
        # 검색 결과가 있다면
        results = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
        if results:   

            # 시장 정보 클릭
            driver.execute_script('return document.querySelector(".place_bluelink").click()')
            time.sleep(2)

            driver.switch_to.default_content()
            # 프레임 변경
            driver.switch_to.frame("entryIframe") # 프레임 변경안됨
            time.sleep(2)
            
            # 시장 평균 별점 수집
            try:
                rating = driver.find_element(By.CSS_SELECTOR, 'span.PXMot.LXIwF').text
                print('시장 평균 별점 : ', rating)
                rev_list.append(
                    [df['mID'][i],
                     rating
                     ]
                )
            except:
                pass
                    
    except: 
        pass
        
driver.close()

# 별점 리스트 → 데이터프레임 변환 및 병합
ratings_df = pd.DataFrame(rev_list, columns=["mID", "rating"])
df = pd.merge(df, ratings_df, on="mID", how="left")

[1] 경기도 가평군 설악눈메골시장 검색 중...
[2] 경기도 가평군 가평잣고을시장 검색 중...
[3] 경기도 가평군 청평여울시장 검색 중...
[4] 경기도 고양시 원당시장 검색 중...
시장 평균 별점 :  별점
3.77
[5] 경기도 고양시 능곡시장 검색 중...
[6] 경기도 고양시 일산시장 검색 중...
시장 평균 별점 :  별점
4.31
[7] 경기도 고양시 덕이동패션1번지 검색 중...
[8] 경기도 과천시 과천새서울프라자시장 검색 중...
[9] 경기도 과천시 제일쇼핑시장 검색 중...
[10] 경기도 광명시 광명새마을시장 검색 중...
시장 평균 별점 :  별점
4.22
[11] 경기도 광명시 광명전통시장 검색 중...
시장 평균 별점 :  별점
4.37
[12] 경기도 광주시 경안시장 검색 중...
시장 평균 별점 :  별점
4.27
[13] 경기도 구리시 구리전통시장 검색 중...
시장 평균 별점 :  별점
4.33
[14] 경기도 군포시 군포역전시장 검색 중...
시장 평균 별점 :  별점
4.27
[15] 경기도 군포시 산본시장 검색 중...
시장 평균 별점 :  별점
4.21
[16] 경기도 김포시 통진시장 검색 중...
[17] 경기도 김포시 양곡시장 검색 중...
[18] 경기도 김포시 양곡오라니장터시장 검색 중...
[19] 경기도 남양주시 남양주장현 전통시장 검색 중...
[20] 경기도 동두천시 동두천제일시장 검색 중...
[21] 경기도 동두천시 중앙시장 검색 중...
[22] 경기도 동두천시 동두천큰시장 검색 중...
[23] 경기도 동두천시 세아프라자상가 검색 중...
[24] 경기도 부천시 조공시장 검색 중...
[25] 경기도 부천시 강남시장 검색 중...
시장 평균 별점 :  별점
4.31
[26] 경기도 부천시 신흥시장 검색 중...
시장 평균 별점 :  별점
4.41
[27] 경기도 부천시 부천상동시장 검색 중...
시장 평균 별점 :  별점
4.28
[28] 경기도 부천시 역곡남부시장 검색 중...
시장 평

In [18]:
(df['rating_x'] == "").sum()

np.int64(1296)

In [19]:
(df['rating_y'] == "").sum()

np.int64(0)

In [20]:
df

,mID,naverURL,rating_x,rating_y
0,경기도 가평군 설악눈메골시장,https://pcmap.place.naver.com/restaurant/14187...,,NaN
1,경기도 가평군 가평잣고을시장,https://pcmap.place.naver.com/restaurant/38738...,,NaN
2,경기도 가평군 청평여울시장,https://pcmap.place.naver.com/restaurant/11536...,,NaN
3,경기도 고양시 원당시장,https://pcmap.place.naver.com/restaurant/32892...,,별점\n3.77
4,경기도 고양시 능곡시장,https://pcmap.place.naver.com/restaurant/16192...,,NaN
...,...,...,...,...
1291,제주특별자치도 제주시 신산시장,https://pcmap.place.naver.com/restaurant/17092...,,NaN
1292,제주특별자치도 제주시 함덕오일시장,https://pcmap.place.naver.com/restaurant/13345...,,NaN
1293,제주특별자치도 제주시 한림민속오일시장(한림오일시장),https://pcmap.place.naver.com/restaurant/13224...,,NaN
1294,제주특별자치도 제주시 한림매일시장,https://pcmap.place.naver.com/restaurant/13347...,,NaN


In [24]:
df = df.drop(columns=['rating_x'])      # rating_x 컬럼 제거
df = df.rename(columns={'rating_y': 'rating'})  # rating_y → rating 으로 이름 변경
df = df.reset_index(drop=True) # 인덱스 초기화

In [25]:
df.columns

Index(['mID', 'naverURL', 'rating'], dtype='object')

In [27]:
print(df[df['mID'] == '서울특별시 서대문구 유진상가']['rating'])

232    별점\n4
Name: rating, dtype: object


In [28]:
df.to_csv('./markets_data/markets_rating.csv', encoding='utf-8-sig', index=False)

In [ ]:
# # 리뷰 데이터 병합
# df[['평점', '리뷰수']] = df['시장ID'].apply(lambda x: pd.Series(get_review_data(x)))

In [ ]:
# # 저장
# conn = sqlite3.connect('market_review.db')
# market_df.to_sql('market_info', conn, if_exists='replace', index=False)
# conn.close()